# Download responses and analyze them with DuckDB

In [1]:
!pip install --upgrade pip citric duckdb-engine faker jupysql sqlalchemy --quiet

In [2]:
from pathlib import Path

from faker import Faker

import citric

## Create citric client

In [3]:
# Use your own server's parameters here
client = citric.Client(
    "http://localhost:8001/index.php/admin/remotecontrol",
    "iamadmin",
    "secret",
)

## Load survey file

In [4]:
with Path("../../examples/survey.lss").open("rb") as f:
    survey_id = client.import_survey(f)

## Generate fake response data

In [5]:
fake = Faker()

fake_responses = []

for _ in range(100):
    start_date = fake.date_time_this_month()
    submit_date = fake.date_time_between_dates(start_date)

    response = {
        "G01Q01": fake.text(max_nb_chars=100),
        "G01Q02": fake.random_int(1, 5),
        "token": fake.sha1()[:5],
        "ipaddr": fake.ipv4(),
        "startdate": start_date.isoformat(),
        "submitdate": submit_date.isoformat(),
    }

    fake_responses.append(response)

len(fake_responses)

100

## Activate survey and load fake responses

In [6]:
client.activate_survey(
    survey_id,
    user_activation_settings={
        "datestamp": True,
    },
)
client.activate_tokens(survey_id)

result = client.add_responses(survey_id, fake_responses)
len(result)

100

## Export responses to a CSV file

In [7]:
with Path("responses.csv").open("wb") as file:
    file.write(
        client.export_responses(
            survey_id,
            file_format="csv",
        )
    )

## Import the CSV into DuckDB

In [8]:
%load_ext sql

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

In [9]:
%sql duckdb://

Connecting to 'duckdb://'

In [10]:
%sql CREATE TABLE responses AS SELECT * FROM 'responses.csv'

Running query in 'duckdb://'

Count


## Query the responses with DuckDB

In [11]:
%sql SELECT * FROM responses LIMIT 5

Running query in 'duckdb://'

id,submitdate,lastpage,startlanguage,seed,token,startdate,datestamp,G01Q01,G01Q02,G02Q03,G02Q03[filecount]
1,2024-02-04 08:26:31.140133,None,en,None,d360f,2024-02-01 17:42:02.678612,2024-02-08 17:19:08,Should check nor will. Every difficult thousand vote provide information nice.,3,None,None
2,2024-02-05 10:16:08.647063,None,en,None,c164f,2024-02-03 07:43:22.637163,2024-02-08 17:19:09,Dog tree rise born career amount. Few energy market since method could film.,4,None,None
3,2024-02-07 20:59:08.821572,None,en,None,2730f,2024-02-06 15:59:43.136551,2024-02-08 17:19:09,Set study station ten. Responsibility early better big how which. Him truth art.,3,None,None
4,2024-02-07 19:26:01.977188,None,en,None,43736,2024-02-07 04:15:00.716972,2024-02-08 17:19:09,Vote see per former message present. Apply drop citizen site yard image group window.,5,None,None
5,2024-02-03 00:05:17.846386,None,en,None,bfd64,2024-02-01 01:10:01.432302,2024-02-08 17:19:09,Truth woman hot much official rather old. Few paper song yard woman likely.,3,None,None


### Show tally for multiple choice question

In [12]:
%sql SELECT G01Q02, count(1) as TOTAL FROM responses GROUP BY G01Q02

Running query in 'duckdb://'

G01Q02,TOTAL
1,20
2,15
3,26
4,19
5,20


### Show which respondents took the longest to submit their responses

In [13]:
%%sql
SELECT token, submitdate - startdate AS duration
FROM responses
ORDER BY 2 DESC LIMIT 10

Running query in 'duckdb://'

token,duration
01a8a,"7 days, 2:47:48.802061"
8462d,"6 days, 14:02:59.801133"
9a07a,"5 days, 14:45:12.970604"
aa7b1,"5 days, 10:55:18.878968"
65aaf,"5 days, 7:17:34.454648"
d2b4f,"5 days, 1:28:12.931126"
137cd,"4 days, 21:42:49.472567"
4398c,"4 days, 18:26:43.167388"
332f3,"4 days, 13:44:53.925752"
b6750,"4 days, 9:41:03.360606"
